In [7]:
import requests
import json
import os
import cmlapi

USER_QUESTION = "What is Cloudera Machine Learning?"  # Update this with your own!
TEMPERATURE = 0.1
TOKEN_COUNT = 250

client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))

## Here we assume that only one model has been deployed in the project, if this is not true this should be adjusted (this is reflected by the placeholder 0 in the array)
model = client.list_models(project_id=os.getenv("CDSW_PROJECT_ID"))
selected_model = model.models[0]

## Save the access key for the model to the environment variable of this project
MODEL_ACCESS_KEY = selected_model.access_key

MODEL_ENDPOINT = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model?accessKey=")
MODEL_ENDPOINT = MODEL_ENDPOINT + MODEL_ACCESS_KEY



In [13]:
llama_sys = f"<s>[INST]You are a helpful, respectful and honest assistant. If you are unsure about an answer, truthfully say \"I don't know\"."    
question_and_context = f"{llama_sys} \n[INST] {USER_QUESTION} [/INST]"

os.environ["MODEL_REQUEST_TIMEOUT_SECONDS"] = "60"

try:
    # Build a request payload for CML hosted model
    data={ "request": {"prompt":question_and_context,"temperature":TEMPERATURE,"max_new_tokens":TOKEN_COUNT,"repetition_penalty":1.0} }
    
    r = requests.post(MODEL_ENDPOINT, data=json.dumps(data), headers={'Content-Type': 'application/json'})
    
    # Logging
    print(f"Request: {data}")
    print(f"Response: {r.json()}")
    
    response = str(r.json()['response']['prediction']['response'])[len(question_and_context)-6:]
        
    print(response)
    
except Exception as e:
    print(e)

# Check if the request was successful
if r.status_code == 200:
    print(f"Success! Received response: {r.json()}")
else:
    print(f"Failed to make request. Status code: {r.status_code}, Response: {r.text}")


Request: {'request': {'prompt': '<s>[INST]You are a helpful, respectful and honest assistant. If you are unsure about an answer, truthfully say "I don\'t know". \n[INST] What is Cloudera Machine Learning? [/INST]', 'temperature': 0.1, 'max_new_tokens': 250, 'repetition_penalty': 1.0}}
Response: {'errors': {'error while calling model': 'context deadline exceeded'}}
'response'
Failed to make request. Status code: 500, Response: {"errors":{"error while calling model":"context deadline exceeded"}}
